
<div>
<img src="https://clms-prod.eea.europa.eu/static/media/EU-Cop-logo.4120d6e3.svg" width="200"/>
</div>
<div>
<img src="https://clms-prod.eea.europa.eu/static/media/ccl-icon-land-text.bfaba89b.svg" width="200"/>
</div>

***
The goal of this notebook is to help you kickstart your project and to provide an example on how to download a dataset. 
You can see all the API documentation here: <br> https://eea.github.io/clms-api-docs/ <br>
You can view information regarding the Machine2Machine download here: <br>https://clms-prod.eea.europa.eu/en/how-to-guides/download-spatial-data/m2m-download
***

## Sign up and token declaration

* Go to https://clms-prod.eea.europa.eu/en  
* Register using EU login
* After signing in to the portal navigate to 'My profile' under your username
* Create a new token, press Copy to clipboard and paste it in a .json file called 'my_saved_key.json' in the same folder as this notebook



In [9]:
import json
import jwt
import time
import requests

In [10]:
import pandas as pd 

In [11]:


# Load saved key from filesystem
service_key = json.load(open('my_saved_key.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')


***

## Find the items that can be downloaded


In [4]:
url_downloadable_items = "https://clms-prod.eea.europa.eu/api/@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=dataset_full_format&&metadata_fields=dataset_download_information"
headers = {'Accept': 'application/json'}
response_downloadable_items = requests.request("GET", url_downloadable_items, headers=headers)

json_downloadable_items = json.loads(response_downloadable_items.text)

for i in range(len(json_downloadable_items['items'])):
    print('Product information:')
    print('\n')
    print(3*' '+'Product title: "'+ json_downloadable_items['items'][i]['title'].replace(':','')+'"')
    print(3*' '+'UID: "'+json_downloadable_items['items'][i]['UID'].replace(' ','')+'"')
    print(3*' '+'Product link: '+ json_downloadable_items['items'][i]['@id']+'')
    print('\n')
    for j in range(len(json_downloadable_items['items'][i]['dataset_download_information']['items'])):
        
        print('Download option for "'+json_downloadable_items['items'][i]['title'].replace(':','')+'":')
        df_downloadable_items = pd.json_normalize(json_downloadable_items['items'][i]['dataset_download_information']['items'][j])
        display(df_downloadable_items)
        print(2*'\n')

    print('\n')
    print(100*'_')       

Product information:


   Product title: "Present Land Cover"
   UID: "e27dbc8330084b58bb5e282231fca74d"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/lclcc-hot-spots/present_land_cover




____________________________________________________________________________________________________
Product information:


   Product title: "Land Cover Change"
   UID: "9827d711d3e148aabccc76dd501a4b86"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/lclcc-hot-spots/land_cover_change




____________________________________________________________________________________________________
Product information:


   Product title: "Key Landscapes for Conservation (KLC) borders"
   UID: "6c47ce296686442a80fd2b0e925d9882"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/lclcc-hot-spots/klc_borders




____________________________________________________________________________________________________
Product information:


   Product title: "Satellite 

,@id,collection,full_format,full_path,full_source,name,wekeo_choices
0,f328dbcc-9069-4240-b8bb-6d5df918671a,100m,Geotiff,v3.0.1/2019,LANDCOVER,RASTER,







____________________________________________________________________________________________________
Product information:


   Product title: "Land Cover 100m collection 3 epoch 2018 Globe"
   UID: "a08da3f50c8d4b38bec35b831382162b"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2018-globe


Download option for "Land Cover 100m collection 3 epoch 2018 Globe":


,@id,collection,full_format,full_path,full_source,name,wekeo_choices
0,00058a15-088e-4461-9c35-b6d4467f34ca,100m,Geotiff,v3.0.1/2018,LANDCOVER,RASTER,







____________________________________________________________________________________________________
Product information:


   Product title: "Land Cover 100m collection 3 epoch 2017 Globe"
   UID: "884325e8871049848a423de02d3cac4d"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2017-globe


Download option for "Land Cover 100m collection 3 epoch 2017 Globe":


...






____________________________________________________________________________________________________
Product information:


   Product title: "Imperviousness Density 2012"
   UID: "d1393a40eba0424bbb68bd3143b8d983"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/high-resolution-layer-imperviousness/high-resolution-layer-imperviousness-density-2012


Download option for "Imperviousness Density 2012":


,@id,collection,full_format,full_path,full_source,name,wekeo_choices
0,9250f7bf-319c-45a1-8718-74f02c13508e,20m,Geotiff,H:\High_Resolution_Layers\High_Resolution_Laye...,EEA,RASTER,





Download option for "Imperviousness Density 2012":


,@id,collection,full_format,full_path,full_source,name
0,55b3d769-edda-47e7-82b5-3b382b8506c1,100m,Geotiff,H:\High_Resolution_Layers\High_Resolution_Laye...,EEA,RASTER







____________________________________________________________________________________________________


### Download example dataset

You are welcome to specify the configuration of your data request by using the json variable. 
If the request is successful, then a task ID will be created under your account and when the data extraction is ready to be downloaded you will receive an email with the download link. <br> <br>
Further documentation can be found here: https://eea.github.io/clms-api-docs/download.html#request-the-download

In [16]:
url_download_item = "https://clms-prod.eea.europa.eu/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json','Authorization': f'Bearer {grant}'}
json={'Datasets': [{'DatasetID': 'a8d945f0edd143a0a5240c28bafa23da', 'DatasetDownloadInformationID': 'f328dbcc-9069-4240-b8bb-6d5df918671a', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITC11'}, {'DatasetID': 'a8d945f0edd143a0a5240c28bafa23da', 'DatasetDownloadInformationID': 'f328dbcc-9069-4240-b8bb-6d5df918671a', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITC1'}]}
response_download_item = requests.post(url_download_item, headers=headers,json=json)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

Response: <Response [401]>
Message with TaskID:
{
  "message": "You are not authorized to access this resource.",
  "type": "Unauthorized"
}


***

## Search prepackaged files that can be downloaded

In [13]:
url_downloadable_prepackaged = "https://clms-prod.eea.europa.eu/api/@search?portal_type=DataSet&metadata_fields=UID&metadata_fields=downloadable_files"
headers = {'Accept': 'application/json'}
response_downloadable_prepackaged = requests.request("GET", url_downloadable_prepackaged, headers=headers)

json_downloadable_prepackaged = json.loads(response_downloadable_prepackaged.text)

for i in range(len(json_downloadable_prepackaged['items'])):
    print('Product information:')
    print('\n')
    print(3*' '+'Product title: "'+ json_downloadable_prepackaged['items'][i]['title'].replace(':','')+'"')
    print(3*' '+'UID: "'+json_downloadable_prepackaged['items'][i]['UID'].replace(' ','')+'"')
    print(3*' '+'Product link: '+ json_downloadable_prepackaged['items'][i]['@id']+'')
    print('\n')
    
    for j in range(len(json_downloadable_prepackaged['items'][i]['downloadable_files']['items'])):

        print('Download option for "'+json_downloadable_prepackaged['items'][i]['title'].replace(':','')+'":')
        df_downloadable_prepackaged = pd.json_normalize(json_downloadable_prepackaged['items'][i]['downloadable_files']['items'][j])
        display(df_downloadable_prepackaged)
        print(2*'\n')

    print('\n')
    print(100*'_') 

Product information:


   Product title: "Present Land Cover"
   UID: "e27dbc8330084b58bb5e282231fca74d"
   Product link: https://clms-prod.eea.europa.eu/api/en/products/lclcc-hot-spots/present_land_cover


Download option for "Present Land Cover":


,@id,area,file,format,path,resolution,size,source,title,type,version,year
0,af1c0570-def6-46a0-8c9b-c6a801fad5bf,Caribbean–Dichotomous Reference LandCover,CAR_01_lc_a,shapefile,https://land.copernicus.eu/global/hsm/php/cope...,30m,,HOTSPOT,Caribbean–Dichotomous Reference LandCover,Vector,,





Download option for "Present Land Cover":


,@id,area,file,format,path,resolution,size,source,title,type,version,year
0,d5229b87-4bd9-422a-8ac1-80880f3d24f8,Caribbean–Modular Reference LandCover,CAR_01_lc_b,shapefile,https://land.copernicus.eu/global/hsm/php/cope...,30m,,HOTSPOT,Caribbean–Modular Reference LandCover,Vector,,





Download option for "Present Land Cover":


...




Download option for "Imperviousness Density 2012":


,@id,area,file,format,path,resolution,size,source,type,version,year
0,6c53bfa6-19df-402a-ae3d-a3c52cf1de4f,,IMD_2012_020m_eu_03035_d03_Full,Geotiff,H:\High_Resolution_Layers\High_Resolution_Laye...,,,EEA,,,





Download option for "Imperviousness Density 2012":


,@id,area,file,format,path,resolution,size,source,type,version,year
0,f998c847-38b5-4414-be8a-d5b853af67f0,,IMD_2012_100m_eu_03035_d03_Full,Geotiff,H:\High_Resolution_Layers\High_Resolution_Laye...,,,EEA,,,







____________________________________________________________________________________________________


### Download example prepackaged file

You are welcome to specify the configuration of your data request by using the json variable. 
If the request is successful, then a task ID will be created under your account and when the data extraction is ready to be downloaded you will receive an email with the download link. <br> <br>
Further documentation can be found here: https://eea.github.io/clms-api-docs/download.html#download-prepackaged-files

In [17]:
url_download_prepackaged = "https://clms-prod.eea.europa.eu/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json','Authorization': f'Bearer {grant}'}
json={'Datasets': [{'DatasetID': '0407d497d3c44bcd93ce8fd5bf78596a', 'FileID': '3d9e2413-46ca-4f6e-abf8-b6eb429a4e48'}, {'DatasetID': '0407d497d3c44bcd93ce8fd5bf78596a', 'FileID': '3bfb1a41-8b59-425d-b172-bd44517adbca'}]}
response_download_prepackaged = requests.post(url_download_prepackaged, headers=headers,json=json)
print('Response:',response_download_prepackaged)
print('Message with TaskID:\n'+response_download_prepackaged.text)

Response: <Response [401]>
Message with TaskID:
{
  "message": "You are not authorized to access this resource.",
  "type": "Unauthorized"
}
